In [1]:
# package(s) related to time, space and id
import datetime, time

# package(s) related to time, space and id
import datetime, time

# package(s) related to the simulation
import simpy
import networkx as nx  
import numpy as np
import math

# OpenTNSim
import opentnsim.core as core

# spatial libraries 
import shapely.geometry
import shapely.wkt
import pyproj
import shapely.geometry

# define the coorinate system
geod = pyproj.Geod(ellps="WGS84")

In [2]:
Node = type('Site', (core.Identifiable, core.Log, core.Locatable, core.HasResource), {})
nodes = []
path = []

distances = [550,500,300,500,150,150,500,300,500,550]
coords = []
coords.append([0,0])

for d in range(len(distances)):
    coords.append([pyproj.Geod(ellps="WGS84").fwd(coords[d][0],coords[d][1],90,distances[d])[0],0])

for d in range(len(coords)):
    data_node = {"env": [],
                 "name": "Node " + str(d+1),
                 "geometry": shapely.geometry.Point(coords[d][0], coords[d][1])}
    node = Node(**data_node)
    nodes.append(node)

for i in range(2):
    if i == 0:
        for j in range(len(nodes)-1):
            path.append([nodes[j], nodes[j+1]])
    if i == 1:
        for j in range(len(nodes)-1):
            path.append([nodes[j+1], nodes[j]])

FG = nx.DiGraph()

positions = {}
for node in nodes:
    positions[node.name] = (node.geometry.x, node.geometry.y)
    FG.add_node(node.name, geometry = node.geometry)

for edge in path:
    FG.add_edge(edge[0].name, edge[1].name, weight = 1)

In [3]:
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))

In [4]:
Vessel = type('Vessel', 
              (core.Identifiable, core.Movable, core.HasContainer, core.HasResource, core.Routeable, core.VesselProperties), {})
start_point = 'Node 1' 
end_point = 'Node 11'  

data_vessel_one = {"env": env,
                   "name": "Vessel",
                   "route": nx.dijkstra_path(FG, end_point, start_point, weight='length'),
                   "geometry": FG.nodes[end_point]['geometry'],
                   "capacity": 1_000,
                   "v": 4,
                   "type": 'CEMT - Va',
                   "B": 10,
                   "H_e": 7.10,
                   "H_f": 5.40,
                   "T_e": 1.8,
                   "T_f": 3.5,
                   "L": 135.0}

env.FG = FG
vessels = []

vessel = Vessel(**data_vessel_one)
vessels.append(vessel)

In [5]:
# water level difference
wlev_dif = [np.linspace(0,45000,1000),np.zeros(1000)]
for i in range(len(wlev_dif[0])):
    wlev_dif[1][i] = 2

In [6]:
# lock area parameters
waiting_area_1 = core.IsLockWaitingArea(env = env, nr_resources = 1, priority = True, 
                                        name = 'Volkeraksluizen_1', node = "Node 2")

lineup_area_1 = core.IsLockLineUpArea(env = env, nr_resources = 1, priority = True, 
                                        name = 'Volkeraksluizen_1', node = "Node 3", lineup_length = 300)

lock_1 = core.IsLock(env = env, nr_resources = 100, priority = True, name = 'Volkeraksluizen_1',            
                     node_1 = "Node 5", node_2 = "Node 6", node_3 = "Node 7",
                     lock_length = 300, lock_width = 24, lock_depth = 4.5, 
                     doors_open = 10 * 60, doors_close = 10 * 60, wlev_dif = wlev_dif, disch_coeff = 0.8,
                     grav_acc = 9.81, opening_area = 4.0, opening_depth = 5.0,
                     simulation_start = simulation_start, operating_time = 25*60)

waiting_area_2 = core.IsLockWaitingArea(env = env, nr_resources = 1, priority = True, 
                                        name = "Volkeraksluizen_1", node = "Node 10")

lineup_area_2 = core.IsLockLineUpArea(env = env, nr_resources = 1, priority = True, 
                                      name = "Volkeraksluizen_1", node = "Node 9", lineup_length = 300)

lock_1.water_level = "Node 5"

In [7]:
#location of lock areas in graph
FG.nodes["Node 6"]["Lock"] = [lock_1]

FG.nodes["Node 2"]["Waiting area"] = [waiting_area_1]
FG.nodes["Node 3"]["Line-up area"] = [lineup_area_1]

FG.nodes["Node 10"]["Waiting area"] = [waiting_area_2]
FG.nodes["Node 9"]["Line-up area"] = [lineup_area_2] 

In [8]:
for vessel in vessels:
    vessel.env = env
    env.process(vessel.move())

In [9]:
env.FG = FG
env.run()

77a5f405-c1f9-11eb-ba91-a0afbd9ae9bb Node 11 Node 10
77a5f405-c1f9-11eb-ba91-a0afbd9ae9bb Node 10 Node 9
77a5f405-c1f9-11eb-ba91-a0afbd9ae9bb Node 10 Node 9


AttributeError: 'Vessel' object has no attribute 'lock_name'